In [10]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [11]:
sin = sp.sin
cos = sp.cos
tan = sp.tan
arctan2 = sp.atan2
sp.init_printing()

Definte variables

In [12]:
Vx_ego, Vy_ego, W, phi, zeta = sp.symbols(['Vx_ego', 'Vy_ego', 'W', 'phi', 'zeta']) # ground speed perpendicular to HD, .. parallel to HD, wind speed, heading, wind direction

U_s, U_phi = sp.symbols(['U_s', 'U_phi']) # control signals (RL agent outputs): speed and angular velocity

U_s_dot, U_phi_dot = sp.symbols([r'\dot{U_s}', r'\dot{U_\phi}']) # control signals derivatives

zeta_dot = sp.symbols(r'\dot{\zeta}') # wind direction derivative

Get derivatives of Vx_ego and Vy_ego

In [13]:
W_dot = 0
Vx_wind_ego = W*sin(zeta - phi)  # wind speed perpendicular to HD = W*sin(zeta - phi)
Vy_wind_ego = W*cos(zeta - phi)  # wind speed parallel to HD = W*cos(zeta - phi)
Vx_ego = Vx_wind_ego
Vy_ego = U_s + Vy_wind_ego # ground speed parallel to HD = U_s + W*sin(zeta) 

Vx_ego_dot = sp.diff(Vx_wind_ego, W)*W_dot + sp.diff(Vx_wind_ego, zeta)*zeta_dot + sp.diff(Vx_wind_ego, phi)*U_phi_dot
print(f"Vx_ego_dot:")
display(Vx_ego_dot)

Vy_ego_dot = sp.diff(Vy_ego, U_s)*U_s_dot + sp.diff(Vy_ego, W)*W_dot + sp.diff(Vy_ego, phi)*U_phi_dot + sp.diff(Vy_ego, zeta)*zeta_dot
print(f"Vy_ego_dot:")
display(Vy_ego_dot)
# NOTE: sympy orders terms alphabetically. Cos is even and Sin is odd which explains the output

Vx_ego_dot:


Vy_ego_dot:


full dynamics

In [14]:
f_Vx = sp.Matrix([1, 0, 0, 0, 0])
f_Vy = sp.Matrix([0, 1, 0, 0, 0])
# f_W = sp.Matrix([0, 0, 1, 0, 0]) # wind intensity is constant
f_phi = sp.Matrix([0, 0, 0, 1, 0])
f_zeta = sp.Matrix([0, 0, 0, 0, 1])
full_dynamics = Vx_ego_dot*f_Vx + Vy_ego_dot*f_Vy + U_phi*f_phi + zeta_dot*f_zeta
full_dynamics

# Vx_ego, Vy_ego, W, phi, zeta 

⎡      -W⋅\dot{U_\phi}⋅cos(φ - ζ) + W⋅\dot{\zeta}⋅cos(φ - ζ)      ⎤
⎢                                                                 ⎥
⎢-W⋅\dot{U_\phi}⋅sin(φ - ζ) + W⋅\dot{\zeta}⋅sin(φ - ζ) + \dot{U_s}⎥
⎢                                                                 ⎥
⎢                                0                                ⎥
⎢                                                                 ⎥
⎢                               Uᵩ                                ⎥
⎢                                                                 ⎥
⎣                           \dot{\zeta}                           ⎦

### Observability

In [15]:
state_vars = sp.Matrix([Vx_ego, Vy_ego, W, phi, zeta]) # airspeed of the agent x y, wind speed, heading, wind direction
display(state_vars)


⎡  -W⋅sin(φ - ζ)  ⎤
⎢                 ⎥
⎢Uₛ + W⋅cos(φ - ζ)⎥
⎢                 ⎥
⎢        W        ⎥
⎢                 ⎥
⎢        φ        ⎥
⎢                 ⎥
⎣        ζ        ⎦

In [16]:
h = sp.Matrix([phi, # head direction / apparent airflow angle
               -U_s, #  Cartesian form of apparent wind flow, Y component, parallel to heading. X component is always 0; egocentric apparent wind direction = wind_allo_angle_radians - agent_angle_radians =  np.angle(- state['air_velocity']) - agent_angle_radians 
               arctan2(Vy_ego, Vx_ego) # drift angle, egocentric course direction NOTE: arctan2 may be differen from np.angle used in simulations; also angles in simulation may have been normalized by pi - pay special attention to this
              ])
h

⎡                   φ                   ⎤
⎢                                       ⎥
⎢                  -Uₛ                  ⎥
⎢                                       ⎥
⎣atan2(Uₛ + W⋅cos(φ - ζ), -W⋅sin(φ - ζ))⎦